In [25]:
from data_functions import *

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import statsmodels.formula.api as smf

import random

In [26]:
for_pres = read('texas_president.xlsx')
for_pres = for_pres.rename(index=str, columns={"count": "county"})
for_pres = for_pres.rename(index=str, columns={"Urbanization Status": "urban", "Border Status": "border"})

In [27]:
for_pres.border.value_counts()

Non-Border    222
Border         32
Name: border, dtype: int64

In [28]:
for_house = read('texas_counties.xlsx')
for_house = for_house.rename(index=str, columns={"rep_number": "district_num", "candidate_name": "dem_candidate", "dem_vote_share": "two_party_vtshare", "Urbanization Status": "urban", "Border Status": "border"})
for_house = for_house[for_house.county != 'ALL COUNTIES']
print(for_house.shape)

(308, 7)


In [29]:
counties = list(for_pres.county)

county_nums = {}
val = 1
for c in counties:
    county_nums[c] = val
    val += 1
    
for_house['county_num'] = for_house.county.apply(lambda x: county_nums[x])
for_house['office'] = 'house'
for_house = for_house[(for_house.two_party_vtshare != 1) & (for_house.two_party_vtshare != 0)]

## COUNTY INFO

In [30]:
county_info = for_pres[['county', 'county_num', 'urban', 'border']]
county_info['urban'] = county_info.urban.apply(lambda x: 1 if x == 'Urban' else 0)
county_info['border'] = county_info.urban.apply(lambda x: 1 if x == 'Border' else 0)

In [31]:
cd = read('demo.csv')
cd = cd.drop(['Unnamed: 0','FIPS','Age'], axis=1)[1:]
cd.County = cd.County.apply(lambda x: x.split(" ")[0])
cd = cd[['County', 'Total', 'Total Female', 'Hispanic Total']]
cd['percent_female'] = cd['Total Female'] / cd['Total']
cd['percent_hispanic'] = cd['Hispanic Total'] / cd['Total']

cd.columns = ['county', 'population', 'Total Female', 'Hispanic Total', 'percent_female','percent_hispanic']
cd = cd[['county', 'population', 'percent_female','percent_hispanic']]
county_info = county_info.merge(cd)

vc = read('vets_and_crimes.xlsx')
vc.county = vc.county.apply(lambda x: x.upper())
county_info = county_info.merge(vc)
county_info = county_info.drop(['crimes_per_100k'], axis=1)

## NEXT INFO

In [32]:
pres_info = for_pres[['office', 'dem_candidate', 'two_party_vtshare', 'candidate_male', 'candidate_white', 'candidate_hispanic']]
#pres_info = pres_candidates.groupby(['dem_candidate','office']).median()
#pres_info = pres_candidates.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [33]:
candidate_df = pd.read_csv('/Users/saptarshighose/Downloads/Texas-Blue-master 3/house_candidates_to_add_info.csv')

In [34]:
candidate_df.head()

,dem_candidate,office,district_num,med_two_party_vtshare,pro_gun_control,pro_choice,pro_climate,candidate_male,candidate_hispanic,candidate_female
0,Adam P. Bell,house,3,0.564941,1.0,1.0,1.0,1,0,0
1,Al Green,house,9,0.802775,1.0,1.0,1.0,1,0,0
2,Bill Bradshaw,house,12,0.167534,NaN,NaN,NaN,1,0,0
3,Eddie Bernice Johnson,house,30,0.804179,1.0,1.0,1.0,0,0,1
4,Eric Mauck,house,26,0.320650,1.0,0.0,1.0,1,0,0


In [37]:
for_house.head(20)

,county,district_num,dem_candidate,two_party_vtshare,pro_gun,pro_choice,climate_pro,county_num,office
0,ANGELINA,1,Shirley McKellar,0.324669,NaN,NaN,NaN,3,house
1,GREGG,1,Shirley McKellar,0.380268,NaN,NaN,NaN,92,house
2,HARRISON,1,Shirley McKellar,0.361176,NaN,NaN,NaN,102,house
3,NACOGDOCHES,1,Shirley McKellar,0.422490,NaN,NaN,NaN,174,house
4,PANOLA,1,Shirley McKellar,0.214388,NaN,NaN,NaN,183,house
5,RUSK,1,Shirley McKellar,0.247915,NaN,NaN,NaN,201,house
6,SABINE,1,Shirley McKellar,0.156508,NaN,NaN,NaN,202,house
7,SAN AUGUSTINE,1,Shirley McKellar,0.344308,NaN,NaN,NaN,203,house
8,SHELBY,1,Shirley McKellar,0.239284,NaN,NaN,NaN,210,house
9,SMITH,1,Shirley McKellar,0.345705,NaN,NaN,NaN,212,house


In [38]:
for_house = for_house.drop(['pro_gun', 'pro_choice','climate_pro'], axis=1)

In [43]:
for_house = for_house.merge(candidate_df, how = 'left', on = 'dem_candidate')

In [44]:
for_house.head()

,county,district_num_x,dem_candidate,two_party_vtshare,county_num,office_x,office_y,district_num_y,med_two_party_vtshare,pro_gun_control,pro_choice,pro_climate,candidate_male,candidate_hispanic,candidate_female
0,ANGELINA,1,Shirley McKellar,0.324669,3,house,house,1,0.286292,1.0,1.0,1.0,0,0,1
1,GREGG,1,Shirley McKellar,0.380268,92,house,house,1,0.286292,1.0,1.0,1.0,0,0,1
2,HARRISON,1,Shirley McKellar,0.361176,102,house,house,1,0.286292,1.0,1.0,1.0,0,0,1
3,NACOGDOCHES,1,Shirley McKellar,0.422490,174,house,house,1,0.286292,1.0,1.0,1.0,0,0,1
4,PANOLA,1,Shirley McKellar,0.214388,183,house,house,1,0.286292,1.0,1.0,1.0,0,0,1


In [45]:
for_house = for_house.drop(['office_y', 'district_num_y'], axis=1)

In [46]:
for_house.head()

,county,district_num_x,dem_candidate,two_party_vtshare,county_num,office_x,med_two_party_vtshare,pro_gun_control,pro_choice,pro_climate,candidate_male,candidate_hispanic,candidate_female
0,ANGELINA,1,Shirley McKellar,0.324669,3,house,0.286292,1.0,1.0,1.0,0,0,1
1,GREGG,1,Shirley McKellar,0.380268,92,house,0.286292,1.0,1.0,1.0,0,0,1
2,HARRISON,1,Shirley McKellar,0.361176,102,house,0.286292,1.0,1.0,1.0,0,0,1
3,NACOGDOCHES,1,Shirley McKellar,0.422490,174,house,0.286292,1.0,1.0,1.0,0,0,1
4,PANOLA,1,Shirley McKellar,0.214388,183,house,0.286292,1.0,1.0,1.0,0,0,1


In [47]:
county_info.head()

,county,county_num,urban,border,population,percent_female,percent_hispanic,veterans_per_1000,crimes_per_1000
0,ANDERSON,1,0,0,58305,0.392213,0.176400,59.55,30.616
1,ANDREWS,2,0,0,17829,0.498289,0.531550,46.46,31.301
2,ANGELINA,3,0,0,90652,0.504953,0.226493,66.85,36.680
3,ARANSAS,4,1,0,25234,0.506975,0.274828,114.69,45.775
4,ARCHER,5,1,0,9123,0.504439,0.090650,76.21,11.122


In [48]:
for_house = for_house.merge(county_info, how = 'left', on = 'county')

In [49]:
for_house.head()

,county,district_num_x,dem_candidate,two_party_vtshare,county_num_x,office_x,med_two_party_vtshare,pro_gun_control,pro_choice,pro_climate,...,candidate_hispanic,candidate_female,county_num_y,urban,border,population,percent_female,percent_hispanic,veterans_per_1000,crimes_per_1000
0,ANGELINA,1,Shirley McKellar,0.324669,3,house,0.286292,1.0,1.0,1.0,...,0,1,3.0,0.0,0.0,90652.0,0.504953,0.226493,66.85,36.680
1,GREGG,1,Shirley McKellar,0.380268,92,house,0.286292,1.0,1.0,1.0,...,0,1,92.0,1.0,0.0,125907.0,0.505564,0.213594,70.82,48.014
2,HARRISON,1,Shirley McKellar,0.361176,102,house,0.286292,1.0,1.0,1.0,...,0,1,102.0,0.0,0.0,68716.0,0.502532,0.145265,69.43,26.614
3,NACOGDOCHES,1,Shirley McKellar,0.422490,174,house,0.286292,1.0,1.0,1.0,...,0,1,174.0,0.0,0.0,65596.0,0.515016,0.209220,52.96,29.661
4,PANOLA,1,Shirley McKellar,0.214388,183,house,0.286292,1.0,1.0,1.0,...,0,1,183.0,0.0,0.0,24145.0,0.499110,0.098820,68.79,32.602


In [50]:
for_house.to_csv('data_for_model.csv')